In [1]:
import cv2 

stream = cv2.VideoCapture(0)
stream.release()

In [2]:
import torch
import requests
from torch import hub # Hub contains other models like FasterRCNN
model = torch.hub.load( \
                      'ultralytics/yolov5', \
                      'yolov5s', \
                      pretrained=True)

/Users/pouya/opt/anaconda3/envs/cv2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /Users/pouya/.cache/torch/hub/ultralytics_yolov5_master


NameError: name 'j' is not defined